<a href="https://colab.research.google.com/github/faisalaldwaish/Sentiment-Analysis-of-Restaurant-Reviews/blob/main/full_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install transformers gradio





In [11]:
pip install fuzzywuzzy[speedup]


In [12]:
import pandas as pd

# Load the uploaded file
file_path = 'Restaurant_reviews.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
df.head(150)

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN
...,...,...,...,...,...,...,...,...
145,Paradise,Aijaz,Had a good time and best service...near place ...,5,"2 Reviews , 167 Followers",5/20/2019 20:51,0,NaN
146,Paradise,Prashant Kumar K,Taste of the biriyani is well defined with the...,5,"1 Review , 1 Follower",5/19/2019 20:02,0,NaN
147,Paradise,Saikrishna CS,Biryani taste is good. Service is also good. T...,5,"1 Review , 2 Followers",5/19/2019 17:59,0,NaN
148,Paradise,Amar Stark,I like the way the staff treats the customers....,5,"2 Reviews , 1 Follower",5/19/2019 14:11,0,NaN


In [13]:
# استيراد المكتبات اللازمة
import pandas as pd  # للتعامل مع البيانات
from transformers import pipeline  # لاستخدام نموذج التصنيف العاطفي
import gradio as gr  # لإنشاء واجهات المستخدم التفاعلية
import seaborn as sns  # لرسم المخططات البيانية
import matplotlib.pyplot as plt  # لإنشاء الرسومات البيانية
from fuzzywuzzy import fuzz  # لمقارنة النصوص بطريقة تطابق غير دقيقة (fuzzy matching)

# تحميل بيانات مراجعات المطاعم من ملف CSV
reviews_df = pd.read_csv('Restaurant_reviews.csv')

# تحميل نموذج التصنيف العاطفي المدرب مسبقاً باستخدام مكتبة Hugging Face
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# تعريف دالة لتصنيف مراجعة المستخدم
def classify_review(user_review):
    try:
        # التحقق من أن المراجعة غير فارغة
        if not user_review.strip():
            return "Please enter a valid review."  # إرجاع رسالة خطأ إذا كانت المراجعة فارغة

        best_match = None  # متغير لتخزين أفضل تطابق للمراجعة
        best_score = 0  # متغير لتخزين أعلى درجة تطابق

        # مقارنة مراجعة المستخدم مع كل مراجعة في قاعدة البيانات باستخدام التطابق الغير دقيق
        for _, row in reviews_df.iterrows():
            if pd.isna(row['Review']):  # تخطي الصفوف التي تحتوي على مراجعات فارغة
                continue

            # حساب درجة التطابق بين المراجعة المدخلة والمراجعة من قاعدة البيانات
            score = fuzz.token_sort_ratio(user_review.lower(), str(row['Review']).lower())
            if score > best_score:  # إذا كانت درجة التطابق أعلى من السابقة، يتم تحديث أفضل تطابق
                best_score = score
                best_match = row

        # إذا كانت درجة التطابق أكثر من 80%
        if best_score > 80:
            # جلب تقييم المراجعة المتطابقة
            rating = best_match['Rating']
            # تحديد نوع المراجعة بناءً على التقييم (إيجابية أو سلبية)
            rating_based_classification = f"Positive review based on rating: {rating}" if int(rating) >= 4 else f"Negative review based on rating: {rating}"

            # إجراء تحليل عاطفي للمراجعة باستخدام نموذج التصنيف العاطفي
            sentiment_result = sentiment_model(user_review)[0]
            sentiment = sentiment_result['label']  # استخراج النتيجة العاطفية
            confidence = sentiment_result['score']  # استخراج نسبة الثقة

            # صياغة النتيجة النهائية مع الثقة والدرجة العاطفية
            sentiment_based_classification = f"Model prediction: {sentiment} with confidence: {confidence:.2f}"

            # إرجاع النتيجة النهائية التي تشمل التصنيف بناءً على التقييم والتصنيف العاطفي
            return f"{rating_based_classification}\n{sentiment_based_classification}\nMatching Score: {best_score}%"
        else:
            # إذا لم يتم العثور على تطابق جيد
            return "Review not found in the dataset."

    except Exception as e:
        # التعامل مع أي خطأ غير متوقع
        return f"An error occurred: {str(e)}"


In [15]:

# تعريف دالة لرسم توزيع التقييمات في مجموعة البيانات
def plot_rating_distribution():
    plt.figure(figsize=(8, 6))  # إنشاء شكل بياني بحجم 8x6
    sns.countplot(x='Rating', data=reviews_df, order=[1, 2, 3, 4, 5])  # رسم توزيع التقييمات
    plt.title('Distribution of Ratings')  # إضافة عنوان للرسم البياني
    plt.xlabel('Rating')  # إضافة تسمية للمحور الأفقي
    plt.ylabel('Count')  # إضافة تسمية للمحور الرأسي
    plt.tight_layout()  # ترتيب العناصر داخل الشكل
    return plt.gcf()  # إرجاع الشكل البياني الحالي





In [16]:
# إنشاء واجهة Gradio لتصنيف المراجعات
review_interface = gr.Interface(
    fn=classify_review,  # الدالة التي سيتم استدعاؤها لتصنيف المراجعات
    inputs=gr.Textbox(lines=2, placeholder="Enter your review here", label="Reviews"),  # صندوق نصي لإدخال المراجعة
    outputs="text",  # نوع الإخراج سيكون نصاً
    title="Review Classifier Based on Rating and Hugging Face Model",  # عنوان الواجهة
    description="Enter a restaurant review. The system will classify it based on the dataset rating and use a sentiment analysis model."  # وصف الواجهة
)

# إنشاء واجهة Gradio لرسم توزيع التقييمات
plot_interface = gr.Interface(
    fn=plot_rating_distribution,  # الدالة التي سيتم استدعاؤها لرسم التوزيع
    inputs=[],  # لا يوجد مدخلات
    outputs="plot",  # نوع الإخراج سيكون رسماً بيانياً
    title="Rating Distribution",  # عنوان الواجهة
    description="Shows the distribution of ratings in the dataset."  # وصف الواجهة
)

# دمج الواجهتين باستخدام تبويبات
tabbed_interface = gr.TabbedInterface([review_interface, plot_interface], ["Review Classifier", "Rating Distribution"])

# إطلاق واجهات Gradio
tabbed_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c4d1257bbd485ee779.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
